# 경찰청 5차본 데이터셋 전처리 (2023.12.08 기준)

In [2]:
import os
import shutil
import pandas as pd
from PIL import Image
import time
import logging
import sys

## 1. 파일 경로 지정

In [19]:
# 경찰청 데이터셋 압축 파일 저장한 위치를 입력하세요
dataset_dir = os.path.expanduser('~/Documents')

# 경찰청 데이터셋 압축 파일 이름을 입력하세요
police_dataset_fname = 'policelab5차본(1-4차본 포함)_수정본' # 버전에 따라 이름이 다를 수 있음

police_dataset_dir = os.path.join(dataset_dir, police_dataset_fname)

# 경찰청 데이터셋 정보 엑셀 파일(유류, 등록 이미지와 문양 정보 파일 경로가 정리되어 있음) 이름을 입력하세요
db_info_name = 'DB정리_20231122_포공.xlsx' # 버전 및 날짜에 따라 파일 이름이 다를 수 있음
db_info_dir = os.path.join(police_dataset_dir, db_info_name)


## 2. 이미지 확장자 통일 확인
- 5차본 기준. 버전에 따라 폴더 구조가 달라질 수 있으니 직접 열어서 확인해보길 바랍니다.
- 등록 이미지와 유류 이미지 모두 파일 확장자가 'png'라고 되어있다고 가정합니다.
- 파일 확장자가 대문자로 적혀있을 경우 다르게 인식한 후 모두 소문자로 통일합니다.

In [20]:
def unify_file_extension(imgs_dir, targ_ext, inplace=True):
    # Verify if targ_ext is a valid image extension.
    if targ_ext.lower() not in ['jpg', 'jpeg', 'png', 'bmp', 'tiff', 'gif']:
        raise ValueError(f"Invalid image extension: {targ_ext}")
    
    # log in  sys.stdout
    logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s %(message)s', datefmt='%H:%M:%S')
    logger = logging.getLogger("unify_file_extension")
    
    img_list = sorted(os.listdir(imgs_dir))
    for img_fname in img_list:
        cur_ext = img_fname.split('.')[-1]
        if cur_ext.lower() != targ_ext:
            raise ValueError(f"Invalid image extension: {cur_ext}")
        elif (cur_ext != targ_ext) and (cur_ext.lower() == targ_ext) :
            if inplace:
                new_img_fname = img_fname.rstrip(cur_ext) + targ_ext
                os.rename(os.path.join(imgs_dir, img_fname), os.path.join(imgs_dir, new_img_fname))
                logger.info(f"Renamed {img_fname} to {new_img_fname}.")
            else:
                # make a new file if inplace is True instead of renaming the file.
                shutil.copyfile(os.path.join(imgs_dir, img_fname), os.path.join(imgs_dir, img_fname.rstrip(cur_ext) + targ_ext))
                logger.info(f"Copied {img_fname} into {new_img_fname}.")
    return True

In [21]:
# 현장(유류) 이미지 파일의 저장 경로
query_imgs_dir = os.path.join(police_dataset_dir, 'pair', 'registration', 'bottom')
# 등록 이미지 파일의 저장 경로
ref_imgs_dir = os.path.join(police_dataset_dir, 'pair', 'Leaving', 'image')

target_extension = 'png'
unify_file_extension(imgs_dir=query_imgs_dir, targ_ext=target_extension, inplace=True)
unify_file_extension(imgs_dir=ref_imgs_dir, targ_ext=target_extension, inplace=True)

True

## 3. label table 생성
- 각 유류 이미지(query image)마다 매칭되는 등록 이미지(reference image / label) 파일 이름을 담고 있는 csv 파일 생성
- 등록 이미지 파일 경로에 유류 이미지 파일 경로가 잘못 들어가있는 경우 에러 메시지를 출력한 후 종료

In [29]:
def create_label_table(db_info_fname, save_fname, query_colname, ref_colname, change_ref_name=True):
    
    df = pd.read_excel(db_info_fname, sheet_name='Sheet1', engine='openpyxl')
    label_table = {"query": [], "ref": []}
    
    ref_tag = 'registration'
    query_tag = 'Leaving'
    
    logger = logging.getLogger("create_label_table")
    
    for idx, row in df.iterrows():
        # check if image file paths are correctly matched with categories(query vs. reference) in excel file.
        if query_tag not in row[query_colname]:
            raise ValueError(f"Query image path is not matched with the category: {idx}th row: {row[query_colname]}")
        
        if ref_tag not in row[ref_colname]:
            if change_ref_name:
                ref_img_name = row['등록밑창이미지이름'] + '.png'
            else:
                raise ValueError(f"Reference image path is not matched with the category: {idx}th row: {row[ref_colname]}")
        
        query_img_name = row[query_colname].split('\\')[-1]
        ref_img_name = row[ref_colname].split('\\')[-1]
        
        label_table["query"].append(query_img_name)
        label_table["ref"].append(ref_img_name)

    label_table_df = pd.DataFrame.from_dict(label_table, orient='columns')
    label_table_df.to_csv(save_fname, index=False)
    logger.info(f"Saved label table to {save_fname}.")

In [33]:
# DB info 엑셀 파일 이름을 입력하세요
db_info_fname = os.path.join(police_dataset_dir, 'DB정리_20231122_포공.xlsx')

query_colname = '유류족적이미지경로'
ref_colname = '등록이미지경로'

# label table을 저장하고자 하는 현재 경로를 지정하세요
working_dir = '.'

time_stamp = time.strftime("%m%d_%H%M", time.localtime())
lable_table_fname = os.path.join(working_dir, f'label_table_{time_stamp}.csv')

In [34]:
create_label_table(db_info_fname, lable_table_fname, query_colname, ref_colname)

11:24:39 Saved label table to ./label_table_1211_1124.csv.


In [36]:
# check if the image files in label_table actually exist.
def check_existing_data(label_table_dir, query_img_dir, ref_img_dir, save_fname):
    df = pd.read_csv(label_table_dir)
    
    query_imgs = os.listdir(query_img_dir)
    ref_imgs = os.listdir(ref_img_dir)
    
    missing_imgs = {"query": [], "ref": [], "missing":[]}
    logger = logging.getLogger("check_existing_data")
    
    for _, row in df.iterrows():
        query_img_name = row['query']
        ref_img_name = row['ref']
        
        if query_img_name not in query_imgs:
            missing_imgs["query"].append(query_img_name)
            missing_imgs["ref"].append(ref_img_name)
            missing_imgs["missing"].append("query")
            
            logger.info(f"{query_img_name} in label_table does NOT exist in real query image folder: {query_img_dir}.")
        
        if ref_img_name not in ref_imgs:
            missing_imgs["query"].append(query_img_name)
            missing_imgs["ref"].append(ref_img_name)
            missing_imgs["missing"].append("ref")
            
            logger.info(f"{ref_img_name} in label_table does NOT exist in real ref image folder: {ref_img_dir}.")
    
    missing_imgs = pd.DataFrame.from_dict(missing_imgs, orient='columns')
    missing_imgs.to_csv(save_fname, index=True)
    logger.info(f"Saved list of missing images to {save_fname}.")
   
    return True

In [37]:
check_existing_data(label_table_dir=lable_table_fname, \
        query_img_dir='query', \
        ref_img_dir='ref', \
        save_fname=os.path.join(working_dir, f'missing_imgs_{time_stamp}.csv'))

11:25:22 B265225.png in label_table does NOT exist in real ref image folder: ref.
11:25:22 B265225.png in label_table does NOT exist in real ref image folder: ref.
11:25:22 B2551162.png in label_table does NOT exist in real ref image folder: ref.
11:25:22 00001-21-0113_3.png in label_table does NOT exist in real query image folder: query.
11:25:23 Saved list of missing images to ./missing_imgs_1211_1124.csv.


True